In [1]:
from collections import defaultdict
import folium
from folium.features import DivIcon
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
import pandas as pd
import random
import requests
import time

In [2]:
def get_travel_distance(coordinates):
    """
    Given an excel file with coordinates, this function generates a distance matrix
    """
    API_KEY = <API_KEY>
    # api-endpoint
    satirNo = 0
    distances = {}
    durations = {}
    for from_index, from_row in coordinates.iterrows():
        for to_index, to_row in coordinates.iterrows():
            from_coordinate = str(from_row["latitude"]) + "," + str(from_row["longitude"])
            to_coordinate = str(to_row["latitude"]) + "," + str(to_row["longitude"])
            URL = "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins=" + from_coordinate +\
                "&destinations=" + to_coordinate + "&travelMode=driving&key=" + API_KEY
            r = requests.get(url=URL)
            data = r.json()
            distances[from_index, to_index] = data["resourceSets"][0]["resources"][0]["results"][0]["travelDistance"]
            durations[from_index, to_index] = data["resourceSets"][0]["resources"][0]["results"][0]["travelDuration"]
    return distances, durations

In [3]:
coordinates = pd.read_excel("coordinates.xlsx")
distances, durations = get_travel_distance(coordinates)

In [4]:
depot = (coordinates["latitude"].mean(), coordinates["longitude"].mean())
mapit = folium.Map(location=[depot[0],depot[1]], zoom_start=10)
for _, coordinate in coordinates.iterrows():
    folium.Marker(location=[coordinate[0], coordinate[1]], radius=8).add_to(mapit)
mapit

C:\Users\cihan.aksop\AppData\Local\Temp\ipykernel_11120\1325978152.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  folium.Marker(location=[coordinate[0], coordinate[1]], radius=8).add_to(mapit)


In [5]:
data = {}
data["time_matrix"] = [[durations[(i,j)] for j in range(len(coordinates))] for i in range(len(coordinates))]
data["time_windows"] = coordinates.apply(lambda x: (x["Begin Time (h)"], x["End Time (h)"]), axis=1)
data["depot"] = 0
manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), 1, data["depot"])
routing = pywrapcp.RoutingModel(manager)
def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["time_matrix"][from_node][to_node]
transit_callback_index = routing.RegisterTransitCallback(time_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
time = "Time"
routing.AddDimension(
    transit_callback_index,
    2, # h allow waiting time
    21, # h maximum time per vehicle
    False,
    time
)
time_dimension = routing.GetDimensionOrDie(time)
for location_idx, time_window in enumerate(data["time_windows"]):
    if location_idx == data["depot"]:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(int(time_window[0]), int(time_window[1]))
for location_index in range(routing.nodes()):
    index = manager.NodeToIndex(location_index)
    if routing.IsEnd(index) or routing.IsStart(index):
        continue
    routing.AddToAssignment(time_dimension.TransitVar(index))
    routing.AddToAssignment(time_dimension.SlackVar(index))

index = routing.Start(data["depot"])
time_dimension.CumulVar(index).SetRange(int(data["time_windows"][data["depot"]][0]), int(data["time_windows"][data["depot"]][1]))
routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(0)))
routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(0)))

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy().PATH_CHEAPEST_ARC
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
#search_parameters.solution_limit = 200
#search_parameters.time_limit.seconds = 5
solution = routing.SolveWithParameters(search_parameters)

print("Solver status: ", routing.status())
route=[]
if solution:
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        index = solution.Value(routing.NextVar(index))
    print("->".join(str(r) for r in route))

Solver status:  7
0->8->5->3->1->9->7->6->4->2


In [6]:
m = folium.Map(location=[depot[0],depot[1]], zoom_start=10)
my_PolyLine=folium.PolyLine(locations=coordinates.loc[route][["latitude", "longitude"]],weight=5)
for _, coordinate in coordinates.iterrows():
    folium.Marker(location=[coordinate.iloc[0], coordinate.iloc[1]], icon=DivIcon(
        html='<b>('+str(coordinate.iloc[2]) + "-" + str(coordinate.iloc[3]) +')<b>',
        ), radius=8).add_to(m)
m.add_child(my_PolyLine)